In [ ]:
%pip install Shapely==1.6.4.post2
#%pip install pycristoforo

In [ ]:
# import pycristoforo as pyc

# country_1 = pyc.get_shape("Poland")
# country_2 = pyc.get_shape("Spain")

# points_1 = pyc.geoloc_generation(country_1, 30, "Poland")
# points_2 = pyc.geoloc_generation(country_2, 30, "Spain")

# print(points_1[0])

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [20.976563177308364, 53.22449990827475]}, 'properties': {'point': 1, 'country': 'Poland'}}


In [ ]:

import numpy as np

spain = {"min_lat": 38, "max_lat": 43, "min_lon": -6, "max_lon": -1}
poland = {"min_lat": 51, "max_lat": 54, "min_lon": 15, "max_lon": 23}

countries = [spain, poland]
num_points_class = 30

points = []

for idx, country in enumerate(countries):
  lats = np.random.uniform(country["min_lat"], country["max_lat"], (num_points_class, 1))
  lons = np.random.uniform(country["min_lon"], country["max_lon"], (num_points_class, 1))
  heads = np.random.uniform(0, 360, (num_points_class, 1))
  label = np.zeros((num_points_class, 1))
  label[:,:] = idx
  locs = np.hstack((lats, lons, heads, label))
  points.append(locs)

points = np.vstack(points)


In [ ]:
import requests
from google.colab import userdata
from PIL import Image
import io

#gets images 
ims = []
class_names = ["spain", "poland"]

for p in points:
  url = "https://maps.googleapis.com/maps/api/streetview?size=600x300&location="+str(p[0])+","+str(p[1])+"&heading="+str(p[2])+"&radius=25000&source=outdoor&key=" + userdata.get('StreetAPIKey')
  response = requests.get(url)
  in_memory_file = io.BytesIO(response.content)
  im = Image.open(in_memory_file)
  w, h = im.size
  im = im.crop((0, 0, w, h - 25))
  ims.append((im, p[3]))

for idx, (im, label) in enumerate(ims):
  im.save("datasets/spain_poland/"+class_names[int(label)]+str(idx)+".jpg")

In [ ]:
for idx, (im, label) in enumerate(ims):
  im.save("drive/MyDrive/spain_poland/"+class_names[int(label)]+str(idx)+".jpg")

In [ ]:
import requests
import googlemaps
import os
import pandas as pd
import random

#Storing latitude/longitude of pictures so we dont pull the same one twice
lat_long = []
random.seed(4)
# api_key = Use personal key here

# Reading api key + cities to pull data for
parent_folder = os.path.dirname(os.path.dirname(__name__))
google_api_key = api_key
cities = pd.read_csv(parent_folder + "Data//CityList.csv")
gmaps = googlemaps.Client(key=google_api_key)


In [ ]:
def query_random_location(gmaps, location, parent_folder, lat_long,name = "", min_distance = 0.005):
    #One degree Latitude = 64 miles, taking 1/100 of that
    random_multiplier = 0.05

    #Generates random value in range [-0.005, 0.005]
    random_lat = (random.random()-0.5) * random_multiplier
    random_long = (random.random()-0.5) * random_multiplier
    original_lat, original_long = list(location)[0], list(location)[1]
    
    #Calculating new latitude and longitude values
    new_lat, new_long = round(original_lat + random_lat,6), round(original_long + random_long,6)

    #Ensuring new coordinates sufficiently far from old oness
    while (round(new_lat,3), round(new_long,3)) in guessed_coordinates:
        print('redo', [new_lat, new_long])
        random_lat = (random.random()-0.5) * random_multiplier
        random_long = (random.random()-0.5) * random_multiplier
        new_lat, new_long = round(original_lat + random_lat,6), round(original_long + random_long,6)

    guessed_coordinates.append((round(new_lat,3), round(new_long,3)))

    #Adding coordinate set to previously called list
    lat_long.append((round(new_lat, 4), round(new_long, 4)))
    new_location = (new_lat, new_long)

    



In [ ]:
def query_random_location(gmaps, location, parent_folder, lat_long,name = "", min_distance = 0.005):

    """
    Takes input city coordinates, adds a random element, and pulls an image from this location
    ------------------
    INPUT:
        gmaps: Google maps object
        location: tuple containing latitude and longitude
        parent_folder: str containing reference to parent folder
        lat_long: list containing all previously called coordinates
        min_distance: float containing minimum distance allowable between new old/new coordiantes
    ------------------
    OUTPUT:
        None
    """

    #One degree Latitude = 64 miles, taking 1/100 of that
    random_multiplier = 0.05

    #Generates random value in range [-0.005, 0.005]
    random_lat = (random.random()-0.5) * random_multiplier
    random_long = (random.random()-0.5) * random_multiplier
    original_lat, original_long = list(location)[0], list(location)[1]
    
    #Calculating new latitude and longitude values
    new_lat, new_long = round(original_lat + random_lat,6), round(original_long + random_long,6)

    #Ensuring new coordinates sufficiently far from old oness
    while (round(new_lat,3), round(new_long,3)) in guessed_coordinates:
        print('redo', [new_lat, new_long])
        random_lat = (random.random()-0.5) * random_multiplier
        random_long = (random.random()-0.5) * random_multiplier
        new_lat, new_long = round(original_lat + random_lat,6), round(original_long + random_long,6)

    guessed_coordinates.append((round(new_lat,3), round(new_long,3)))

    #Adding coordinate set to previously called list
    lat_long.append((round(new_lat, 4), round(new_long, 4)))
    new_location = (new_lat, new_long)

    #Calling functions
    try:
        city, address = convert_long_lat_to_address(gmaps,new_location)
        pull_image(gmaps, address,parent_folder,str(new_location), name)
    except:
        print("No Image for Location")

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import random
import time

import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

img_files = os.listdir('drive/MyDrive/spain_poland/')
img_labels = list(map(lambda x: 0 if 'spain' in x else 1, img_files))
img_files = list(map(lambda x: 'drive/MyDrive/spain_poland/'+x, img_files))

X_train, X_test, y_train, y_test = train_test_split(img_files, img_labels, test_size=0.2, random_state=42)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# preprocessing of images
class GeoDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        super().__init__()
        self.paths = image_paths
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform

    def __len__(self): return self.len

    def __getitem__(self, index):
        path = self.paths[index]
        image = Image.open(path).convert('RGB')
        image = self.transform(image)
        label = self.labels[index]
        return (image, label)


train_ds = GeoDataset(X_train, y_train, transform)
test_ds = GeoDataset(X_test, y_test, transform)

train_loader = DataLoader(train_ds)
test_loader = DataLoader(test_ds)



In [ ]:
class ConvNet(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
    self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

    # after the above convolutions on a 600x275 image with 3 channels,
    # we should have a 149x68 image with 64 channels
    self.fc1 = nn.Linear(in_features= 64 * 149 * 68, out_features=500)
    self.fc2 = nn.Linear(in_features=500, out_features=50)
    self.fc3 = nn.Linear(in_features=50, out_features=2)

  def forward(self, X):

      X = F.relu(self.conv1(X))
      #X = F.max_pool2d(X, 2)

      X = F.relu(self.conv2(X))
      #X = F.max_pool2d(X, 2)

      X = F.relu(self.conv3(X))
      #X = F.max_pool2d(X, 2)

      X = X.view(X.shape[0], -1)
      X = F.relu(self.fc1(X))
      X = F.relu(self.fc2(X))
      X = self.fc3(X)

      return X


In [ ]:
model = ConvNet()

losses = []
accuracies = []
epochs = 8
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(epochs):

  epoch_loss = 0
  epoch_accuracy = 0

  for X, y in train_loader:
    optimizer.zero_grad()

    preds = model(X)
    loss = loss_fn(preds, y)

    loss.backward()
    optimizer.step()

    accuracy = ((preds.argmax(dim=1) == y).float().mean())
    epoch_accuracy += accuracy
    epoch_loss += loss
    print('.', end='', flush=True)

  epoch_accuracy = epoch_accuracy/len(train_loader)
  accuracies.append(epoch_accuracy)
  epoch_loss = epoch_loss / len(train_loader)
  losses.append(epoch_loss)

  print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

  with torch.no_grad():

    test_epoch_loss = 0
    test_epoch_accuracy = 0

    for test_X, test_y in test_loader:

      test_preds = model(test_X)
      test_loss = loss_fn(test_preds, test_y)

      test_epoch_loss += test_loss
      test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
      test_epoch_accuracy += test_accuracy

    test_epoch_accuracy = test_epoch_accuracy/len(test_loader)
    test_epoch_loss = test_epoch_loss / len(test_loader)

    print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))



................................................
 --- Epoch: 0, train loss: 1.1933, train acc: 0.4792, time: 345.3464891910553
Epoch: 0, test loss: 0.6802, test acc: 0.6667, time: 349.7984585762024

................................................
 --- Epoch: 1, train loss: 0.7614, train acc: 0.5208, time: 658.8543744087219
Epoch: 1, test loss: 0.6728, test acc: 0.6667, time: 660.7695319652557

................................................
 --- Epoch: 2, train loss: 0.7826, train acc: 0.4583, time: 959.3032774925232
Epoch: 2, test loss: 0.6747, test acc: 0.6667, time: 962.321771144867

................................................
 --- Epoch: 3, train loss: 0.7825, train acc: 0.4792, time: 1284.8746178150177
Epoch: 3, test loss: 0.6883, test acc: 0.3333, time: 1287.3642344474792

................................................
 --- Epoch: 4, train loss: 0.7340, train acc: 0.6042, time: 1610.53945851326
Epoch: 4, test loss: 0.6787, test acc: 0.6667, time: 1613.0534069538116

....